In [2]:
pip install pytrends 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install fastapi


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Mini Tutorial: Uso de la API de Google Trends

1. **Configuración y Autenticación:**
   - Obtén credenciales de la API de Google Trends.
   - Autentica tus solicitudes utilizando estas credenciales.

2. **Definir Parámetros:**
   - Especifica el tema, las categorías/temas y los períodos de tiempo de tu interés.

3. **Realizar Solicitudes a la API:**
   - Envía solicitudes HTTP a la API de Google Trends con los parámetros definidos.
   - Si la solicitud falla debido a limitaciones de velocidad u otros problemas, maneja la situación de manera adecuada:
     - Intenta nuevamente la solicitud.
     - Si falla de nuevo, reintenta con una estrategia de espera (por ejemplo, espera 5 minutos).
     - Intenta un máximo de tres veces.
     - Si todavía falla, espera un periodo más largo (por ejemplo, 10 minutos) antes de intentar por cuarta vez.

4. **Procesamiento de Datos:**
   - Una vez que recibas una respuesta exitosa, procesa los datos según sea necesario.
   - Ignora el primer índice si es necesario (puedes crear una función para esto más adelante).

5. **Manejo de Errores:**
   - Implementa un manejo robusto de errores para capturar cualquier error inesperado y registrarlos para su depuración.

6. **Optimización:**
   - Optimiza tu código para la eficiencia, ya que obtener datos de la API puede tomar diferentes cantidades de tiempo según el tema.
   - Considera implementar solicitudes paralelas u otras optimizaciones ...

Siguiendo estos pasos y manejando los problemas potenciales de manera adecuada, puedes utilizar efectivamente la API de Google Trends en tus aplicaciones.


In [14]:
# el primer endpoint 
from pytrends.request import TrendReq
from fastapi import FastAPI
import pandas as pd
import time
import requests

url = 'https://trends.google.com/trends/'
response = requests.get(url, timeout=10)
app = FastAPI()

country_list = ['AT', 'BE', 'CH', 'DE', 'DK', 'EE', 'GR', 'ES',
       'FI', 'FR', 'HU', 'IE', 'IS', 'IT','LU', 'ME',
       'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI','GB',
        'LT', 'LV']

def obtener_datos_google_trends_mensual(palabra_clave, paises=country_list, fecha_inicio='2014-01-01', fecha_fin='2024-03-31'):
    try:
        pytrends = TrendReq(hl='es', tz=360)
        trends_data = {}
        for pais in paises:
            try:
                pytrends.build_payload(kw_list=[palabra_clave], timeframe=f'{fecha_inicio} {fecha_fin}', geo=pais)
                data = pytrends.interest_over_time()
                if data.empty:
                    raise ValueError(f"No hay datos disponibles para la palabra clave y país '{pais}'.")
                data_mensual = data.resample('M').mean()
                trends_data[pais] = data_mensual.to_dict()
            except Exception as e:
                print(f"Error al obtener datos de Google Trends para el país '{pais}': {e}")
                # Break off the loop if an error occurs for a country
                break
        return trends_data
    except Exception as e:
        print(f"Error al obtener datos de Google Trends: {e}")
        time.sleep(60)
        return obtener_datos_google_trends_mensual(palabra_clave, paises, fecha_inicio, fecha_fin)

@app.get("/google-trends/{topic}")
def google_trends_handler(topic: str, fecha_inicio: str = '2014-01-01', fecha_fin: str = '2024-03-31'):
    trends_data = obtener_datos_google_trends_mensual(topic, country_list, fecha_inicio, fecha_fin)
    return trends_data

#ejemplo del uso 

palabra_clave = "Unemployment"
datos8 = obtener_datos_google_trends_mensual(palabra_clave)
datos8



# in case  If it fails , retry with some backoff strategy (e.g., wait 5minutes). if it brak  at ' At' reduce countries or ignore the first index in the list  (we can later creat a function) and retry 
# it my  take a time it depends on the topic [18,6s to 8 minutes ]
# if retries > 3 it will break but dont worry  
# wait 10 minutes and retry again 
# the 4th time should work perfectlly 

{'AT': {'Unemployment': {Timestamp('2014-01-31 00:00:00', freq='M'): 51.0,
   Timestamp('2014-02-28 00:00:00', freq='M'): 38.0,
   Timestamp('2014-03-31 00:00:00', freq='M'): 49.0,
   Timestamp('2014-04-30 00:00:00', freq='M'): 51.0,
   Timestamp('2014-05-31 00:00:00', freq='M'): 33.0,
   Timestamp('2014-06-30 00:00:00', freq='M'): 64.0,
   Timestamp('2014-07-31 00:00:00', freq='M'): 17.0,
   Timestamp('2014-08-31 00:00:00', freq='M'): 45.0,
   Timestamp('2014-09-30 00:00:00', freq='M'): 30.0,
   Timestamp('2014-10-31 00:00:00', freq='M'): 64.0,
   Timestamp('2014-11-30 00:00:00', freq='M'): 37.0,
   Timestamp('2014-12-31 00:00:00', freq='M'): 45.0,
   Timestamp('2015-01-31 00:00:00', freq='M'): 59.0,
   Timestamp('2015-02-28 00:00:00', freq='M'): 71.0,
   Timestamp('2015-03-31 00:00:00', freq='M'): 54.0,
   Timestamp('2015-04-30 00:00:00', freq='M'): 43.0,
   Timestamp('2015-05-31 00:00:00', freq='M'): 49.0,
   Timestamp('2015-06-30 00:00:00', freq='M'): 78.0,
   Timestamp('2015-07-31

In [19]:
#segundo end point 
import os
import logging
from fastapi import FastAPI, HTTPException
from pytrends.request import TrendReq
import pandas as pd

def clean_dataframe(datos8):
    cleaned_df = datos8.dropna().drop_duplicates()
    return cleaned_df

# Endpoint to clean Google Trends data
@app.get("/clean-google-trends-data/{topic}")
def clean_google_trends_data(topic: str):
    try:

        # Convert to DataFrame
        df = pd.DataFrame.from_dict(datos8)

        # Clean DataFrame
        cleaned_df = clean_dataframe(df)

        # Save cleaned data to CSV
        cleaned_df.to_json(f"{topic}_cleaned_data.json", index=True)

        return {"message": f"Data cleaned and saved successfully"}
    except Exception as e:
        logging.error(f"Error cleaning Google Trends data for category '{topic}': {e}")
        raise HTTPException(status_code=500, detail=f"Error cleaning Google Trends data for category '{topic}'")

